### First Draft: NFL Playoffs - Linear & Logistic Regression Model

##### Linear Model that will predict score difference & Classifier that will predict probability of a Win

This is my the notebook for my first model entirly on my own that isn't for a class so I'm going to be doing a little exploring and it might have parts that don't totally make sense.

#### Brainstorm

To start, I need to get data which means I have to decide what data I want to train and build my model off of. To start, I want to look at overall record, home/away,

#### Importing Libraries
Pandas, numpy, and matplotlib, are what I always import at the start. Will import others later if needed.

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Importing data into pandas data frames

Import into one data frame for plays, import to another for scores. We will use plays to get stats and use scores to track each game.

In [48]:
scores = pd.read_csv('../../data/raw/Season_Scores/2023_scores.csv')
plays = pd.read_csv('../../data/raw/PlaysFolder/2023_plays.csv')

### Cleaning Up

Preseason games are removed from the data frame as it is not an accurate representation of a regular season or playoff game. Bye weeks are also removed. Pro bowl games are removed

In [49]:
is_preseason_scores = scores['Week'].str.contains(r"preseason|hall of fame", case=False, na=False)
is_bye = scores['GameStatus'].str.contains(r"bye", case=False, na=False)
is_superbowl_scores = scores['Week'].str.contains(r"super bowl", case=False, na=False)
is_probowl_scores = scores['Week'].str.contains(r"pro bowl", case=False, na=False)
scores = scores[~is_preseason_scores].copy()
scores = scores[~is_bye].copy()
scores = scores[~is_superbowl_scores].copy()

is_preseason_plays = plays['Week'].str.contains(r"preseason|hall of fame", case=False, na=False)
is_superbowl_plays = plays['Week'].str.contains(r"super bowl", case=False, na=False)
plays = plays[~is_preseason_plays].copy()
plays = plays[~is_superbowl_plays].copy()


/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/274068839.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  scores = scores[~is_bye].copy()
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/274068839.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  scores = scores[~is_superbowl_scores].copy()
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/274068839.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plays = plays[~is_superbowl_plays].copy()


In [50]:
TEAM_NAME_MAP = {
    # Washington
    "Washington Football Team": "Commanders",
    "Football Team": "Commanders",
    "Washington Commanders": "Commanders",
    "Commanders": "Commanders",

    # Optional future-proofing
    "Los Angeles Rams": "Rams",
    "St. Louis Rams": "Rams",
    "Oakland Raiders": "Raiders",
    "Las Vegas Raiders": "Raiders",
    "San Diego Chargers": "Chargers",
    "Los Angeles Chargers": "Chargers",
}

def standardize_team_name(name):
    return TEAM_NAME_MAP.get(name, name)

standardize_team_name('Washington Football Team')  # Example usage

def standardize_team_names_in_df(df, columns):
    for col in columns:
         df[col] = df[col].astype(str).str.strip().replace(TEAM_NAME_MAP)
    return df

scores = standardize_team_names_in_df(scores, ['HomeTeam', 'AwayTeam'])
plays = standardize_team_names_in_df(plays, ['AwayTeam', 'HomeTeam'])

In [51]:
keys = ['Season', 'Week', 'HomeTeam', 'AwayTeam']
games = scores[keys].drop_duplicates().reset_index(drop=True)
games['game_id'] = games.index.astype(int)
games[games['game_id']==1073]

,Season,Week,HomeTeam,AwayTeam,game_id


In [52]:
scores = scores.merge(games, on=keys, how="left")
plays  = plays.merge(games, on=keys, how="left")
scores.groupby('Week').size()

Week
Conference Championships     2
Divisional Playoffs          4
Week 1                      16
Week 10                     14
Week 11                     14
Week 12                     16
Week 13                     13
Week 14                     15
Week 15                     16
Week 16                     16
Week 17                     16
Week 18                     16
Week 2                      16
Week 3                      16
Week 4                      16
Week 5                      14
Week 6                      15
Week 7                      13
Week 8                      16
Week 9                      14
Wild Card                    6
dtype: int64

In [53]:
#  Old way to set game_id but it didn't work
#  keys = ['Season', 'Week', 'HomeTeam', 'AwayTeam']
# plays['game_id'] = plays.groupby(keys).ngroup()
# scores['game_id'] = scores.groupby(keys).ngroup()
# scores['game_id'] = scores['game_id'].astype(int)
scores[scores['game_id']==2].iloc[0]

Season           2023
Week           Week 1
GameStatus      FINAL
Day               SUN
Date            09/10
AwayTeam       Texans
AwayRecord        0-1
AwayScore         9.0
AwayWin           0.0
HomeTeam       Ravens
HomeRecord        1-0
HomeScore        25.0
HomeWin           1.0
AwaySeeding       NaN
HomeSeeding       NaN
PostSeason          0
game_id             2
Name: 2, dtype: object

In [54]:
rushing_plays = plays[plays['PlayOutcome'].str.contains(r"rush|run", case=False, na=False)]
rushing_plays['RushYards'] = rushing_plays['PlayOutcome'].str.extract(r"(-?\d+)\s*Yard")
rushing_plays['RushYards'] = rushing_plays['RushYards'].astype(float)

passing_plays = plays[plays['PlayOutcome'].str.contains(r"pass", case=False, na=False)]
passing_plays['PassYards'] = passing_plays['PlayOutcome'].str.extract(r"(-?\d+)\s*Yard")
passing_plays = passing_plays[~passing_plays['PassYards'].isna()]
passing_plays['PassYards'] = passing_plays['PassYards'].astype(float)

touchdown_plays = plays[plays['PlayOutcome'].str.contains(r"touchdown", case=False, na=False)]

/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/2475392443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rushing_plays['RushYards'] = rushing_plays['PlayOutcome'].str.extract(r"(-?\d+)\s*Yard")
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/2475392443.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rushing_plays['RushYards'] = rushing_plays['RushYards'].astype(float)
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/2475392443.py:6: SettingWithCopy

In [55]:
touchdown_plays['pass'] = touchdown_plays['PlayDescription'].str.contains(r"pass", case=False, na=False).astype(int)
touchdown_plays["td_yards"] = touchdown_plays["PlayDescription"].str.extract(r"for\s+(-?\d+)\s+yard")
touchdown_plays["td_yards"] = touchdown_plays["td_yards"].dropna().astype(int)

/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/1236499397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  touchdown_plays['pass'] = touchdown_plays['PlayDescription'].str.contains(r"pass", case=False, na=False).astype(int)
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_43786/1236499397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  touchdown_plays["td_yards"] = touchdown_plays["PlayDescription"].str.extract(r"for\s+(-?\d+)\s+yard")
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0

In [56]:
rush_td_plays = touchdown_plays[touchdown_plays['pass'] == 0]
pass_td_plays = touchdown_plays[touchdown_plays['pass'] == 1]


In [57]:
#Merge rushing and passing stats per game per team on keys
game_stats_keys = ['game_id', 'HomeTeam', 'TeamWithPossession']
rush = (rushing_plays
        .groupby(game_stats_keys, as_index=False)["RushYards"]
        .sum())

pas = (passing_plays
       .groupby(game_stats_keys, as_index=False)["PassYards"]
       .sum())
rush_td = rush_td_plays.groupby(['game_id', 'HomeTeam', 'TeamWithPossession'], as_index=False)['td_yards'].sum()
pass_td = pass_td_plays.groupby(['game_id', 'HomeTeam', 'TeamWithPossession'], as_index=False)['td_yards'].sum()
non_td_stats = rush.merge(pas, on=game_stats_keys, how="outer")
td_stats = rush_td.merge(pass_td, on=game_stats_keys, how="outer", suffixes=('_RushTD', '_PassTD'))
td_stats = td_stats.fillna(0)
display(td_stats.head(5))
display(non_td_stats.head(5))


,game_id,HomeTeam,TeamWithPossession,td_yards_RushTD,td_yards_PassTD
0,0,Chiefs,Detroit Lions,8.0,9.0
1,0,Chiefs,Kansas City Chiefs,0.0,55.0
2,1,Falcons,Atlanta Falcons,6.0,11.0
3,1,Falcons,Carolina Panthers,0.0,4.0
4,2,Ravens,Baltimore Ravens,8.0,0.0


,game_id,HomeTeam,TeamWithPossession,RushYards,PassYards
0,0,Chiefs,Detroit Lions,97.0,241.0
1,0,Chiefs,Kansas City Chiefs,90.0,221.0
2,1,Falcons,Atlanta Falcons,124.0,104.0
3,1,Falcons,Carolina Panthers,141.0,142.0
4,2,Ravens,Baltimore Ravens,106.0,169.0


In [58]:
non_td_stats[non_td_stats['RushYards'].isna()]

,game_id,HomeTeam,TeamWithPossession,RushYards,PassYards


In [59]:
game_stats = non_td_stats.merge(td_stats, on=game_stats_keys, how="outer")
game_stats = game_stats.fillna(0)
game_stats['TeamWithPossession'] = game_stats['TeamWithPossession'].str.split().str[-1]
game_stats['RushYards'] = game_stats['RushYards'] + game_stats['td_yards_RushTD']
game_stats['PassYards'] = game_stats['PassYards'] + game_stats['td_yards_PassTD']
game_stats.drop(columns=['td_yards_RushTD', 'td_yards_PassTD'], inplace=True)
game_stats[game_stats['game_id']==1073]

,game_id,HomeTeam,TeamWithPossession,RushYards,PassYards


In [60]:
print(game_stats['RushYards'].isna().sum() + game_stats['PassYards'].isna().sum())

0


In [61]:
home_stats = game_stats[game_stats['HomeTeam'] == game_stats['TeamWithPossession']].drop('TeamWithPossession', axis=1)
home_stats = home_stats.rename(columns={'RushYards': 'HomeRushYards','PassYards': 'HomePassYards'})
away_stats = game_stats[game_stats['HomeTeam'] != game_stats['TeamWithPossession']].drop('TeamWithPossession', axis=1)
away_stats = away_stats.rename(columns={'RushYards': 'AwayRushYards', 'PassYards': 'AwayPassYards'})

In [62]:
display(home_stats.head())
display(away_stats.head())

,game_id,HomeTeam,HomeRushYards,HomePassYards
1,0,Chiefs,90.0,276.0
2,1,Falcons,130.0,115.0
4,2,Ravens,114.0,169.0
7,3,Browns,191.0,154.0
8,4,Colts,52.0,232.0


,game_id,HomeTeam,AwayRushYards,AwayPassYards
0,0,Chiefs,105.0,250.0
3,1,Falcons,141.0,146.0
5,2,Ravens,92.0,251.0
6,3,Browns,75.0,82.0
9,4,Colts,102.0,241.0


In [63]:
#home_stats['HomePassYards'].isna().sum() + home_stats['HomeRushYards'].isna().sum() + away_stats['AwayPassYards'].isna().sum() + away_stats['AwayRushYards'].isna().sum()
display(home_stats[(home_stats['HomeTeam']=='Eagles')])
#display(away_stats[away_stats['Season']==2024])

,game_id,HomeTeam,HomeRushYards,HomePassYards
33,16,Eagles,259.0,193.0
114,57,Eagles,104.0,334.0
209,104,Eagles,104.0,301.0
267,133,Eagles,114.0,207.0
355,177,Eagles,187.0,200.0
380,190,Eagles,46.0,326.0
477,238,Eagles,173.0,377.0
499,249,Eagles,91.0,184.0


In [64]:
scores = scores.merge(home_stats[['game_id', 'HomeRushYards', 'HomePassYards']], on=['game_id'], how='left')
scores = scores.merge(away_stats[['game_id', 'AwayRushYards', 'AwayPassYards']], on=['game_id'], how='left')

In [65]:
scores[scores['HomeRushYards'].isna()]['game_id']

Series([], Name: game_id, dtype: int64)

In [66]:
#csv doesn't have stats for this game so I am mannualy inputing them
# home_rush = 131
# home_pass = 270
# away_rush = 56
# away_pass = 108

# gid = 220

# scores.loc[gid, "HomeRushYards"] = home_rush
# scores.loc[gid, "HomePassYards"] = home_pass
# scores.loc[gid, "AwayRushYards"] = away_rush
# scores.loc[gid, "AwayPassYards"] = away_pass

In [67]:
new_game_data = pd.read_csv('../../data/raw/GamesFolder/2023_games.csv')

In [68]:
#old way
# new_game_data = pd.read_csv('NFL-scores-data/GamesFolder/2024_games.csv')

In [69]:
new_game_data['HomeTeam'] = np.where(
    new_game_data['@?'] == '@',
    new_game_data['Loser/tie'],
    new_game_data['Winner/tie']
)
new_game_data['AwayTeam'] = np.where(
    new_game_data['@?'].isna(), 
    new_game_data['Loser/tie'], 
    new_game_data['Winner/tie'])

new_game_data['HomeTeamTO'] = np.where(
    new_game_data['@?'] == '@',
    new_game_data['TOL'],
    new_game_data['TOW']
)

new_game_data['AwayTeamTO'] = np.where(
    new_game_data['@?'].isna(), 
    new_game_data['TOL'], 
    new_game_data['TOW']) 

new_game_data['HomeTeam'] = new_game_data['HomeTeam'].str.split().str[-1]
new_game_data['AwayTeam'] = new_game_data['AwayTeam'].str.split().str[-1]
new_game_data.iloc[58]

Week                              4
Day                             Sun
Date                     2023-10-01
Time                         1:00PM
Winner/tie      Philadelphia Eagles
@?                              NaN
Loser/tie     Washington Commanders
Date.1                     boxscore
PtsW                             34
PtsL                             31
YdsW                            415
TOW                               0
YdsL                            365
TOL                               0
HomeTeam                     Eagles
AwayTeam                 Commanders
HomeTeamTO                        0
AwayTeamTO                        0
Name: 58, dtype: object

In [70]:
new_game_data['Date'] = pd.to_datetime(new_game_data['Date'])
new_game_data['Season'] = new_game_data['Date'].dt.year
new_game_data.loc[new_game_data['Date'].dt.month < 9, 'Season'] -= 1
new_game_data

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,AwayTeam,HomeTeamTO,AwayTeamTO,Season
0,1,Thu,2023-09-07,8:20PM,Detroit Lions,@,Kansas City Chiefs,boxscore,21,20,368,1,316,1,Chiefs,Lions,1,1,2023
1,1,Sun,2023-09-10,1:00PM,Atlanta Falcons,NaN,Carolina Panthers,boxscore,24,10,221,0,281,3,Falcons,Panthers,0,3,2023
2,1,Sun,2023-09-10,1:00PM,Cleveland Browns,NaN,Cincinnati Bengals,boxscore,24,3,350,2,142,0,Browns,Bengals,2,0,2023
3,1,Sun,2023-09-10,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,boxscore,31,21,342,2,280,3,Colts,Jaguars,3,2,2023
4,1,Sun,2023-09-10,1:00PM,Washington Commanders,NaN,Arizona Cardinals,boxscore,20,16,248,3,210,2,Commanders,Cardinals,3,2,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,Division,Sun,2024-01-21,3:00PM,Detroit Lions,NaN,Tampa Bay Buccaneers,boxscore,31,23,391,0,408,2,Lions,Buccaneers,0,2,2023
281,Division,Sun,2024-01-21,6:30PM,Kansas City Chiefs,@,Buffalo Bills,boxscore,27,24,361,1,368,0,Bills,Chiefs,0,1,2023
282,ConfChamp,Sun,2024-01-28,3:00PM,Kansas City Chiefs,@,Baltimore Ravens,boxscore,17,10,319,0,336,3,Ravens,Chiefs,3,0,2023
283,ConfChamp,Sun,2024-01-28,6:30PM,San Francisco 49ers,NaN,Detroit Lions,boxscore,34,31,413,1,442,1,49ers,Lions,1,1,2023


In [71]:
new_game_data["Date"] = (
    new_game_data["Date"]
    .astype(str)
    .str.strip()
)
new_game_data['Date'] = pd.to_datetime(
    new_game_data["Date"],
    format="%Y-%m-%d",
    errors="coerce"   # invalid strings become NaT instead of crashing
)

new_game_data

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,AwayTeam,HomeTeamTO,AwayTeamTO,Season
0,1,Thu,2023-09-07,8:20PM,Detroit Lions,@,Kansas City Chiefs,boxscore,21,20,368,1,316,1,Chiefs,Lions,1,1,2023
1,1,Sun,2023-09-10,1:00PM,Atlanta Falcons,NaN,Carolina Panthers,boxscore,24,10,221,0,281,3,Falcons,Panthers,0,3,2023
2,1,Sun,2023-09-10,1:00PM,Cleveland Browns,NaN,Cincinnati Bengals,boxscore,24,3,350,2,142,0,Browns,Bengals,2,0,2023
3,1,Sun,2023-09-10,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,boxscore,31,21,342,2,280,3,Colts,Jaguars,3,2,2023
4,1,Sun,2023-09-10,1:00PM,Washington Commanders,NaN,Arizona Cardinals,boxscore,20,16,248,3,210,2,Commanders,Cardinals,3,2,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,Division,Sun,2024-01-21,3:00PM,Detroit Lions,NaN,Tampa Bay Buccaneers,boxscore,31,23,391,0,408,2,Lions,Buccaneers,0,2,2023
281,Division,Sun,2024-01-21,6:30PM,Kansas City Chiefs,@,Buffalo Bills,boxscore,27,24,361,1,368,0,Bills,Chiefs,0,1,2023
282,ConfChamp,Sun,2024-01-28,3:00PM,Kansas City Chiefs,@,Baltimore Ravens,boxscore,17,10,319,0,336,3,Ravens,Chiefs,3,0,2023
283,ConfChamp,Sun,2024-01-28,6:30PM,San Francisco 49ers,NaN,Detroit Lions,boxscore,34,31,413,1,442,1,49ers,Lions,1,1,2023


In [72]:
#check to see if new_game_data has any NaN values in HomeTeamTO or AwayTeamTO, should be 0 for both
print(new_game_data[new_game_data['AwayTeamTO'].isna()].shape[0])
print(new_game_data[new_game_data['HomeTeamTO'].isna()].shape[0])

0
0


In [73]:
scores["Date"] = scores["Date"].astype(str).str.strip()

scores["Date"] = pd.to_datetime(
    scores["Season"].astype(str) + "-" + scores["Date"],
    format="%Y-%m/%d",
    errors="coerce"
)
scores["Date"] = pd.to_datetime(scores["Date"], errors="coerce")

# Jan/Feb games are in the next calendar year
mask = scores["Date"].dt.month <= 2
scores.loc[mask, "Date"] = scores.loc[mask, "Date"] + pd.DateOffset(years=1)

In [74]:
new_game_data['Date'].dtype, scores['Date'].dtype

(dtype('<M8[ns]'), dtype('<M8[ns]'))

In [75]:
turnovers = new_game_data[
    ["Season", "Date", "HomeTeam", "AwayTeam", "HomeTeamTO", "AwayTeamTO"]
]

scores = scores.merge(
    turnovers,
    on=["Season", "Date", "HomeTeam", "AwayTeam"],
    how="left"
)
scores.head(20)

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,...,AwaySeeding,HomeSeeding,PostSeason,game_id,HomeRushYards,HomePassYards,AwayRushYards,AwayPassYards,HomeTeamTO,AwayTeamTO
0,2023,Week 1,FINAL,THU,2023-09-07,Lions,1-0,21.0,1.0,Chiefs,...,NaN,NaN,0,0,90.0,276.0,105.0,250.0,1,1
1,2023,Week 1,FINAL,SUN,2023-09-10,Panthers,0-1,10.0,0.0,Falcons,...,NaN,NaN,0,1,130.0,115.0,141.0,146.0,0,3
2,2023,Week 1,FINAL,SUN,2023-09-10,Texans,0-1,9.0,0.0,Ravens,...,NaN,NaN,0,2,114.0,169.0,92.0,251.0,2,1
3,2023,Week 1,FINAL,SUN,2023-09-10,Bengals,0-1,3.0,0.0,Browns,...,NaN,NaN,0,3,191.0,154.0,75.0,82.0,2,0
4,2023,Week 1,FINAL,SUN,2023-09-10,Jaguars,1-0,31.0,1.0,Colts,...,NaN,NaN,0,4,52.0,232.0,102.0,241.0,3,2
5,2023,Week 1,FINAL,SUN,2023-09-10,Buccaneers,1-0,20.0,1.0,Vikings,...,NaN,NaN,0,5,41.0,344.0,88.0,173.0,3,0
6,2023,Week 1,FINAL,SUN,2023-09-10,Titans,0-1,15.0,0.0,Saints,...,NaN,NaN,0,6,59.0,305.0,104.0,198.0,2,3
7,2023,Week 1,FINAL,SUN,2023-09-10,49ers,1-0,30.0,1.0,Steelers,...,NaN,NaN,0,7,56.0,217.0,188.0,235.0,2,1
8,2023,Week 1,FINAL,SUN,2023-09-10,Cardinals,0-1,16.0,0.0,Commanders,...,NaN,NaN,0,8,65.0,217.0,96.0,132.0,3,2
9,2023,Week 1,FINAL,SUN,2023-09-10,Packers,1-0,38.0,1.0,Bears,...,NaN,NaN,0,9,132.0,253.0,76.0,245.0,2,0


In [76]:
#Check for missing turnover data --> shows if merge was successful
scores[["HomeTeamTO", "AwayTeamTO"]].isna().mean()

HomeTeamTO    0.0
AwayTeamTO    0.0
dtype: float64

In [77]:
scores.columns

Index(['Season', 'Week', 'GameStatus', 'Day', 'Date', 'AwayTeam', 'AwayRecord',
       'AwayScore', 'AwayWin', 'HomeTeam', 'HomeRecord', 'HomeScore',
       'HomeWin', 'AwaySeeding', 'HomeSeeding', 'PostSeason', 'game_id',
       'HomeRushYards', 'HomePassYards', 'AwayRushYards', 'AwayPassYards',
       'HomeTeamTO', 'AwayTeamTO'],
      dtype='object')

In [78]:
#time to create rolling windows
#we need to be able to collect data based on team and week rather than home/away so before creating windows_df we are going to create team_games

scores = scores.copy()
scores = scores.sort_values('Date').reset_index()

home_team_games = scores[['Season', 'game_id', 'Week', 'Date', 'HomeTeam', 'AwayTeam', 'HomeRushYards', 'AwayRushYards', 'HomePassYards', 'AwayPassYards', 'HomeTeamTO', 'AwayTeamTO', 'HomeScore', 'AwayScore', 'HomeRecord', 'AwayRecord', 'HomeWin']]
home_team_games = home_team_games.rename(columns={
    'HomeTeam': 'Team',
    'AwayTeam': 'Opponent',
    'HomeRushYards': 'RushYards',
    'AwayRushYards': 'RushYardsAllowed',
    'HomePassYards': 'PassYards',
    'AwayPassYards': 'PassYardsAllowed',
    'HomeTeamTO': 'Giveaways',
    'AwayTeamTO': 'Takeaways',
    'HomeScore': 'PointsScored',
    'AwayScore': 'PointsAllowed',
    'HomeRecord': 'TeamRecord',
    'HomeWin': 'Win'})

away_team_games = scores[['Season', 'game_id', 'Week', 'Date', 'AwayTeam', 'HomeTeam', 'AwayRushYards', 'HomeRushYards', 'AwayPassYards', 'HomePassYards',  'AwayTeamTO', 'HomeTeamTO', 'AwayScore', 'HomeScore', 'AwayRecord', 'HomeRecord', 'AwayWin']]
away_team_games = away_team_games.rename(columns={
    'AwayTeam': 'Team',
    'HomeTeam': 'Opponent',
    'AwayRushYards': 'RushYards',
    'HomeRushYards': 'RushYardsAllowed',
    'AwayPassYards': 'PassYards',
    'HomePassYards': 'PassYardsAllowed',
    'AwayTeamTO': 'Giveaways',
    'HomeTeamTO': 'Takeaways',
    'AwayScore': 'PointsScored',
    'HomeScore': 'PointsAllowed',
    'AwayRecord': 'TeamRecord',
    'HomeRecord': 'OpponentRecord',
    'AwayWin': 'Win'})

home_team_games['HomeFieldAdvantage'] = 1
away_team_games['HomeFieldAdvantage'] = 0

team_games = pd.concat([home_team_games, away_team_games], ignore_index=True)
team_games.sort_values(['Team', 'Date'], inplace=True)
team_games

,Season,game_id,Week,Date,Team,Opponent,RushYards,RushYardsAllowed,PassYards,PassYardsAllowed,Giveaways,Takeaways,PointsScored,PointsAllowed,TeamRecord,AwayRecord,Win,HomeFieldAdvantage,OpponentRecord
298,2023,7,Week 1,2023-09-10,49ers,Steelers,188.0,56.0,235.0,217.0,1,2,30.0,7.0,1-0,NaN,1.0,0,0-1
305,2023,26,Week 2,2023-09-17,49ers,Rams,144.0,102.0,206.0,307.0,0,2,30.0,23.0,2-0,NaN,1.0,0,1-1
32,2023,32,Week 3,2023-09-21,49ers,Giants,130.0,29.0,315.0,152.0,0,1,30.0,12.0,3-0,1-2,1.0,1,NaN
50,2023,61,Week 4,2023-10-01,49ers,Cardinals,124.0,91.0,285.0,265.0,0,0,35.0,16.0,4-0,1-3,1.0,1,NaN
66,2023,76,Week 5,2023-10-08,49ers,Cowboys,181.0,40.0,253.0,161.0,1,4,42.0,10.0,5-0,3-2,1.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,2023,201,Week 14,2023-12-10,Vikings,Raiders,132.0,46.0,146.0,140.0,0,3,3.0,0.0,7-6,NaN,1.0,0,5-8
493,2023,209,Week 15,2023-12-16,Vikings,Bengals,143.0,80.0,303.0,324.0,2,1,24.0,27.0,7-7,NaN,0.0,0,8-6
232,2023,230,Week 16,2023-12-24,Vikings,Lions,17.0,130.0,411.0,272.0,4,1,24.0,30.0,7-8,11-4,0.0,1,NaN
242,2023,255,Week 17,2023-12-31,Vikings,Packers,87.0,178.0,180.0,293.0,2,1,10.0,33.0,7-9,8-8,0.0,1,NaN


In [79]:
#Create rolling windows dataframe
windows = [3, 5, 7]
stats = [
    "RushYards", "RushYardsAllowed",
    "PassYards", "PassYardsAllowed",
    "Giveaways", "Takeaways",
    "PointsScored", "PointsAllowed", "Win"]

for w in windows:
    for s in stats:
        team_games[f"{s}{w}GameAvg"] = team_games.groupby(['Season', 'Team']).shift(1).rolling(window=w, min_periods=w)[s].mean()

rolling_cols = [col for col in team_games.columns if 'Avg' in col]
id_cols = ['Season', 'game_id', 'Team', 'Opponent', 'Date', 'Week', 'HomeFieldAdvantage']
target_cols = ['PointsScored', 'PointsAllowed', 'Win']
team_games.head(8)

,Season,game_id,Week,Date,Team,Opponent,RushYards,RushYardsAllowed,PassYards,PassYardsAllowed,...,Win5GameAvg,RushYards7GameAvg,RushYardsAllowed7GameAvg,PassYards7GameAvg,PassYardsAllowed7GameAvg,Giveaways7GameAvg,Takeaways7GameAvg,PointsScored7GameAvg,PointsAllowed7GameAvg,Win7GameAvg
298,2023,7,Week 1,2023-09-10,49ers,Steelers,188.0,56.0,235.0,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,2023,26,Week 2,2023-09-17,49ers,Rams,144.0,102.0,206.0,307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,2023,32,Week 3,2023-09-21,49ers,Giants,130.0,29.0,315.0,152.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2023,61,Week 4,2023-10-01,49ers,Cardinals,124.0,91.0,285.0,265.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,2023,76,Week 5,2023-10-08,49ers,Cowboys,181.0,40.0,253.0,161.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,2023,83,Week 6,2023-10-15,49ers,Browns,123.0,160.0,125.0,192.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,2023,105,Week 7,2023-10-23,49ers,Vikings,63.0,74.0,272.0,378.0,...,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,2023,119,Week 8,2023-10-29,49ers,Bengals,113.0,134.0,365.0,299.0,...,0.6,136.142857,78.857143,241.571429,238.857143,0.857143,1.714286,28.714286,15.571429,0.714286


In [80]:
team_games = team_games.sort_values(['Season', 'Team', 'Date', 'game_id']).copy()
team_games['game_num'] = team_games.groupby(['Season', 'Team']).cumcount() + 1
team_games.head(10)

,Season,game_id,Week,Date,Team,Opponent,RushYards,RushYardsAllowed,PassYards,PassYardsAllowed,...,RushYards7GameAvg,RushYardsAllowed7GameAvg,PassYards7GameAvg,PassYardsAllowed7GameAvg,Giveaways7GameAvg,Takeaways7GameAvg,PointsScored7GameAvg,PointsAllowed7GameAvg,Win7GameAvg,game_num
298,2023,7,Week 1,2023-09-10,49ers,Steelers,188.0,56.0,235.0,217.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
305,2023,26,Week 2,2023-09-17,49ers,Rams,144.0,102.0,206.0,307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
32,2023,32,Week 3,2023-09-21,49ers,Giants,130.0,29.0,315.0,152.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
50,2023,61,Week 4,2023-10-01,49ers,Cardinals,124.0,91.0,285.0,265.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
66,2023,76,Week 5,2023-10-08,49ers,Cowboys,181.0,40.0,253.0,161.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
370,2023,83,Week 6,2023-10-15,49ers,Browns,123.0,160.0,125.0,192.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
389,2023,105,Week 7,2023-10-23,49ers,Vikings,63.0,74.0,272.0,378.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
119,2023,119,Week 8,2023-10-29,49ers,Bengals,113.0,134.0,365.0,299.0,...,136.142857,78.857143,241.571429,238.857143,0.857143,1.714286,28.714286,15.571429,0.714286,8
429,2023,140,Week 10,2023-11-12,49ers,Jaguars,144.0,49.0,316.0,188.0,...,125.428571,90.000000,260.142857,250.571429,1.142857,1.571429,26.857143,19.000000,0.571429,9
154,2023,159,Week 11,2023-11-19,49ers,Buccaneers,105.0,66.0,333.0,246.0,...,125.428571,82.428571,275.857143,233.571429,1.142857,1.857143,27.428571,16.142857,0.571429,10


In [81]:
team_games = team_games[team_games['game_num'] > 7].copy()
team_games_rolling = team_games[id_cols + rolling_cols + target_cols].copy()

In [82]:
home_feat = team_games_rolling.merge(scores[['Season', 'game_id', 'HomeTeam']], left_on=['Season', 'game_id', 'Team'], right_on=['Season', 'game_id', 'HomeTeam'], how='inner')
home_feat = home_feat.drop('HomeFieldAdvantage', axis=1)
home_feat = home_feat[['Season', 'game_id'] + rolling_cols + target_cols]
home_feat = home_feat.rename(columns={c: f'Home{c}' for c in rolling_cols + target_cols})

away_feat = team_games_rolling.merge(scores[['Season', 'game_id', 'AwayTeam']], left_on=['Season', 'game_id', 'Team'], right_on=['Season', 'game_id', 'AwayTeam'], how='inner')
away_feat = away_feat.drop('HomeFieldAdvantage', axis=1)
away_feat = away_feat[['Season', 'game_id'] + rolling_cols]
away_feat = away_feat.rename(columns={c: f'Away{c}' for c in rolling_cols})

In [83]:
display(home_feat.head())
display(away_feat.head())
print(home_feat.shape)
print(away_feat.shape)

,Season,game_id,HomeRushYards3GameAvg,HomeRushYardsAllowed3GameAvg,HomePassYards3GameAvg,HomePassYardsAllowed3GameAvg,HomeGiveaways3GameAvg,HomeTakeaways3GameAvg,HomePointsScored3GameAvg,HomePointsAllowed3GameAvg,...,HomePassYards7GameAvg,HomePassYardsAllowed7GameAvg,HomeGiveaways7GameAvg,HomeTakeaways7GameAvg,HomePointsScored7GameAvg,HomePointsAllowed7GameAvg,HomeWin7GameAvg,HomePointsScored,HomePointsAllowed,HomeWin
0,2023,119,122.333333,91.333333,216.666667,243.666667,1.666667,2.333333,25.333333,17.000000,...,241.571429,238.857143,0.857143,1.714286,28.714286,15.571429,0.714286,17.0,31.0,0.0
1,2023,159,106.666667,85.666667,317.666667,288.333333,2.000000,2.000000,22.666667,18.666667,...,275.857143,233.571429,1.142857,1.857143,27.428571,16.142857,0.571429,27.0,14.0,1.0
2,2023,202,136.666667,67.000000,289.333333,250.666667,0.333333,1.333333,33.333333,15.333333,...,278.000000,258.428571,1.142857,1.714286,26.428571,17.285714,0.571429,28.0,16.0,1.0
3,2023,239,154.333333,109.333333,309.666667,290.666667,0.666667,1.333333,38.333333,21.333333,...,309.142857,255.000000,0.857143,1.857143,32.000000,17.857143,0.857143,19.0,33.0,0.0
4,2023,269,146.666667,131.666667,278.000000,230.666667,1.666667,1.333333,30.333333,24.000000,...,291.428571,244.714286,1.142857,1.428571,31.285714,19.142857,0.857143,20.0,21.0,0.0


,Season,game_id,AwayRushYards3GameAvg,AwayRushYardsAllowed3GameAvg,AwayPassYards3GameAvg,AwayPassYardsAllowed3GameAvg,AwayGiveaways3GameAvg,AwayTakeaways3GameAvg,AwayPointsScored3GameAvg,AwayPointsAllowed3GameAvg,...,AwayWin5GameAvg,AwayRushYards7GameAvg,AwayRushYardsAllowed7GameAvg,AwayPassYards7GameAvg,AwayPassYardsAllowed7GameAvg,AwayGiveaways7GameAvg,AwayTakeaways7GameAvg,AwayPointsScored7GameAvg,AwayPointsAllowed7GameAvg,AwayWin7GameAvg
0,2023,140,99.666667,122.666667,254.000000,289.666667,2.333333,1.333333,17.000000,24.0,...,0.4,125.428571,90.000000,260.142857,250.571429,1.142857,1.571429,26.857143,19.000000,0.571429
1,2023,166,120.666667,83.000000,338.000000,244.333333,1.000000,2.333333,26.000000,16.0,...,0.4,121.857143,87.714286,278.428571,247.000000,1.142857,2.000000,27.000000,16.428571,0.571429
2,2023,190,136.000000,68.000000,290.000000,204.666667,0.333333,2.666667,30.666667,10.0,...,0.6,126.857143,87.428571,269.285714,234.857143,1.285714,2.285714,26.428571,16.000000,0.571429
3,2023,219,159.333333,63.666667,291.000000,258.333333,1.000000,1.333333,33.666667,16.0,...,1.0,129.000000,73.428571,308.428571,269.428571,1.285714,1.714286,28.000000,16.857143,0.714286
4,2023,251,142.666667,129.666667,317.333333,266.000000,2.333333,1.333333,30.666667,26.0,...,0.8,140.285714,91.285714,305.142857,248.285714,1.142857,1.714286,32.285714,18.142857,0.857143


(171, 32)
(173, 29)


In [84]:
model_df = home_feat.merge(away_feat, on=['Season', 'game_id'], how='inner')
model_df

,Season,game_id,HomeRushYards3GameAvg,HomeRushYardsAllowed3GameAvg,HomePassYards3GameAvg,HomePassYardsAllowed3GameAvg,HomeGiveaways3GameAvg,HomeTakeaways3GameAvg,HomePointsScored3GameAvg,HomePointsAllowed3GameAvg,...,AwayWin5GameAvg,AwayRushYards7GameAvg,AwayRushYardsAllowed7GameAvg,AwayPassYards7GameAvg,AwayPassYardsAllowed7GameAvg,AwayGiveaways7GameAvg,AwayTakeaways7GameAvg,AwayPointsScored7GameAvg,AwayPointsAllowed7GameAvg,AwayWin7GameAvg
0,2023,159,106.666667,85.666667,317.666667,288.333333,2.000000,2.000000,22.666667,18.666667,...,0.2,70.857143,93.857143,235.857143,288.428571,1.142857,1.571429,18.714286,19.857143,0.285714
1,2023,202,136.666667,67.000000,289.333333,250.666667,0.333333,1.333333,33.333333,15.333333,...,0.2,89.285714,139.857143,252.000000,234.428571,1.571429,1.142857,20.000000,26.000000,0.428571
2,2023,239,154.333333,109.333333,309.666667,290.666667,0.666667,1.333333,38.333333,21.333333,...,0.8,182.000000,120.428571,220.285714,223.285714,0.857143,1.714286,30.428571,18.285714,0.857143
3,2023,269,146.666667,131.666667,278.000000,230.666667,1.666667,1.333333,30.333333,24.000000,...,0.8,164.000000,83.000000,272.142857,277.000000,1.000000,0.857143,29.285714,21.857143,0.857143
4,2023,279,140.000000,92.666667,248.666667,192.666667,2.000000,1.000000,22.000000,21.333333,...,0.8,131.142857,117.142857,267.000000,269.000000,0.857143,1.142857,28.571429,22.571429,0.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2023,263,91.666667,103.000000,182.333333,242.666667,0.666667,0.333333,12.000000,21.666667,...,0.2,85.857143,114.142857,276.428571,240.571429,1.571429,1.142857,23.000000,21.857143,0.428571
164,2023,141,93.000000,72.666667,313.666667,256.333333,1.333333,2.000000,25.666667,18.333333,...,0.6,104.571429,124.000000,253.571429,212.285714,1.000000,2.000000,22.714286,19.857143,0.428571
165,2023,179,142.000000,69.000000,246.666667,249.666667,1.666667,1.333333,26.000000,22.666667,...,0.4,159.000000,60.571429,195.571429,247.714286,1.571429,1.571429,22.142857,21.285714,0.428571
166,2023,230,116.000000,79.333333,207.666667,227.000000,2.000000,2.000000,12.333333,13.000000,...,0.6,161.571429,118.285714,273.857143,230.571429,1.857143,0.857143,29.714286,25.714286,0.714286


In [85]:
linear_model_df = model_df.drop(columns=['Season', 'game_id', 'HomeWin'])
linear_design_matrix = linear_model_df.drop(columns=['HomePointsScored','HomePointsAllowed'])
linear_target = linear_model_df['HomePointsScored'] - linear_model_df['HomePointsAllowed']

In [86]:
home_cols = [c for c in linear_design_matrix.columns if c.startswith("Home")]

diff_df = pd.DataFrame(index=linear_design_matrix.index)
for home_col in home_cols:
    away_col = home_col.replace("Home", "Away")
    if away_col in linear_design_matrix.columns:
        diff_name = home_col.replace("Home", "Diff")
        diff_df[diff_name] = linear_design_matrix[home_col] - linear_design_matrix[away_col]

linear_design_matrix = diff_df
linear_design_matrix

,DiffRushYards3GameAvg,DiffRushYardsAllowed3GameAvg,DiffPassYards3GameAvg,DiffPassYardsAllowed3GameAvg,DiffGiveaways3GameAvg,DiffTakeaways3GameAvg,DiffPointsScored3GameAvg,DiffPointsAllowed3GameAvg,DiffWin3GameAvg,DiffRushYards5GameAvg,...,DiffWin5GameAvg,DiffRushYards7GameAvg,DiffRushYardsAllowed7GameAvg,DiffPassYards7GameAvg,DiffPassYardsAllowed7GameAvg,DiffGiveaways7GameAvg,DiffTakeaways7GameAvg,DiffPointsScored7GameAvg,DiffPointsAllowed7GameAvg,DiffWin7GameAvg
0,26.333333,14.000000,57.666667,-44.666667,1.333333,1.000000,-2.333333,-4.333333,0.000000,54.8,...,0.2,54.571429,-11.428571,40.000000,-54.857143,0.000000,0.285714,8.714286,-3.714286,0.285714
1,55.333333,-56.000000,34.333333,9.000000,-1.000000,0.666667,12.000000,-14.333333,1.000000,54.2,...,0.6,32.571429,-51.571429,26.000000,24.000000,-0.428571,0.571429,6.428571,-8.714286,0.142857
2,-38.000000,-11.333333,88.333333,29.666667,0.000000,-0.666667,11.666667,5.333333,0.000000,-20.6,...,0.2,-41.428571,-25.285714,88.857143,31.714286,0.000000,0.142857,1.571429,-0.428571,0.000000
3,-20.666667,57.000000,-16.333333,-56.666667,0.000000,0.333333,2.333333,1.666667,-0.333333,-17.4,...,0.0,-17.857143,10.142857,19.285714,-32.285714,0.142857,0.571429,2.000000,-2.714286,0.000000
4,-8.000000,-7.333333,-45.000000,-77.333333,1.333333,-0.333333,-10.666667,4.333333,-0.666667,14.2,...,-0.2,17.714286,-17.857143,3.857143,-36.142857,0.428571,0.142857,1.714286,-2.428571,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,5.333333,-19.666667,-62.333333,54.000000,-1.333333,-0.333333,-3.000000,4.000000,-0.333333,37.0,...,0.0,21.857143,-9.000000,-69.857143,9.571429,-0.285714,-0.428571,-5.428571,2.000000,-0.142857
164,-33.666667,-70.000000,16.000000,53.666667,0.666667,-0.666667,-3.000000,-6.666667,0.333333,-39.8,...,0.2,-9.428571,-46.857143,35.000000,49.857143,0.428571,-0.428571,0.285714,-0.428571,0.285714
165,-40.000000,-14.333333,62.666667,38.000000,-0.333333,0.000000,6.333333,0.000000,0.333333,-44.0,...,0.4,-62.000000,17.857143,75.142857,-8.857143,-0.285714,0.000000,1.142857,-2.000000,0.285714
166,-41.000000,-33.333333,-14.666667,-20.666667,1.000000,1.000000,-17.000000,-11.333333,-0.333333,-15.8,...,-0.2,-42.142857,-46.285714,-34.571429,6.428571,-0.142857,0.714286,-9.857143,-9.000000,-0.142857


In [87]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [88]:
X = linear_design_matrix.copy()
y = linear_target.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Test Error (MAE): {mae}')
print(f'Test Error (RMSE): {rmse}')

train_mae = mean_absolute_error(y_train, y_pred_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f'Training Error (MAE): {train_mae}')
print(f'Training Error (RMSE): {train_rmse}')

Test Error (MAE): 12.513343744274248
Test Error (RMSE): 15.77924156952442
Training Error (MAE): 9.141418223712298
Training Error (RMSE): 11.485274855500212


In [89]:
model.intercept_

np.float64(3.261194029850746)

In [90]:
import joblib
import os

In [91]:
joblib.dump(model, '../../models/v1_midseason/lin_reg_v1.pkl')
joblib.dump(scaler, '../../models/v1_midseason/scaler_v1.pkl')

['../../models/v1_midseason/scaler_v1.pkl']